In [1]:
!pip install wandb

In [2]:
import wandb
key = input('Enter your API:')
wandb.login(key=key)

Enter your API: 580e769ee2f34eafdded556ce52aaf31c265ad3b


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma23m011 (ma23m011-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import wandb

#  # Sweep configuration for hyperparameter tuning
sweep_config = {
    'name': 'resnet50_hyperparam_sweep-3',# Sweep name
    'method': 'bayes',  # Use Bayesian optimization
    'metric': {
        'name': 'val_acc', # Optimize for validation accuracy
        'goal': 'maximize' # Maximize the metric
    },
    'parameters': {
        'batch_size': {
            'values': [16, 32, 64, 128]  # different batch sizes
        },
        'freeze_up_to': {'values': [0, 3, 5, 7]}, # Freeze first few layers
        'epochs': {
            'values': [5,7, 10,12]  # different epoch values
        },
        'learning_rate': {
            'values': [0.1, 0.01, 0.001]  # different learning rates
        }
    }
}

#  Create Sweep
sweep_id = wandb.sweep(sweep_config, project="DL_A2")

# # Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#  Training Function
def train():
    # Initialize W&B run
    with wandb.init() as run:
        config = run.config

        #  Transforms
        transform = transforms.Compose([
            transforms.Resize((224, 224)), # Resize image
            transforms.RandomResizedCrop(224),# Random crop
            transforms.RandomHorizontalFlip(),# Random horizontal flip
            transforms.ToTensor(),  # Convert to tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], # Normalize
                                 std=[0.229, 0.224, 0.225])
        ])

        #  Load Data
        train_data = ImageFolder('/kaggle/input/inaturalist-12/inaturalist_12K/train', transform=transform)
        val_data = ImageFolder('/kaggle/input/inaturalist-12/inaturalist_12K/val', transform=transform)

        # Create data loaders
        train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=config.batch_size)

        dataloaders = {'train': train_loader, 'val': val_loader} # Store loaders 
        dataset_sizes = {'train': len(train_data), 'val': len(val_data)} # Store dataset sizes

        # Load pretrained ResNet50 model
        model = models.resnet50(pretrained=True)
        num_ftrs = model.fc.in_features # Get number of input features to final layer
        model.fc = nn.Linear(num_ftrs, 10) # Replace final layer for 10 classes

        # Freeze layers up to a certain index
        child_counter = 0
        for child in model.children():
            if child_counter < config.freeze_up_to:
                for param in child.parameters():
                    param.requires_grad = False  #freeze
            else:
                for param in child.parameters():
                    param.requires_grad = True  # Unfreeze
            child_counter += 1
        model=model.to(device)  # Move model to device

        #for param in model.parameters():
            #param.requires_grad = False

        #for param in model.fc.parameters():
            #param.requires_grad = True
        #model = model.to(device)

        # Loss and Optimizer
        criterion = nn.CrossEntropyLoss() # Define loss function
        optimizer = optim.Adam(model.fc.parameters(), lr=config.learning_rate) # Define Optimizer
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning rate scheduler

        # Training Loop
        for epoch in range(config.epochs):
            model.train()  # Set model to training mode
            running_loss = 0.0 # initialize running loss
            running_corrects = 0 #initialize running correct prediction

            for inputs, labels in dataloaders['train']:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad() # Clear gradients
                outputs = model(inputs) # Forward pass
                _, preds = torch.max(outputs, 1) # Get predictions
                loss = criterion(outputs, labels) # Compute loss
                loss.backward() # Backpropagation
                optimizer.step() # Update weights

                running_loss += loss.item() * inputs.size(0) # Total loss
                running_corrects += torch.sum(preds == labels.data) # Count correct

            epoch_loss = running_loss / dataset_sizes['train'] # Calculate average loss  for epoch
            epoch_acc = (running_corrects.double() / dataset_sizes['train']) * 100 # Calculate  accuracy for epoch
            
            # wandb log results
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": epoch_loss,
                "train_acc": epoch_acc
            })
            scheduler.step() # Step the scheduler

        # Validation
        model.eval() # Set model to eval mode
        running_corrects = 0
        with torch.no_grad():
            for inputs, labels in dataloaders['val']:
                # selecting device for speed
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs) # Forward pass
                _, preds = torch.max(outputs, 1) # Get predictions
                running_corrects += torch.sum(preds == labels.data) # Count correct

        val_acc = (running_corrects.double() / dataset_sizes['val']) * 100 # Compute validation accuracy
        wandb.log({"val_acc": val_acc})  # Log validation accuracy to wandb
        print(f"[{run.name}] Validation Accuracy: {val_acc:.2f}%") # Print result

# Sweep Agent (This runs experiments)
wandb.agent(sweep_id, function=train, count=50)


Create sweep with ID: tfufum48
Sweep URL: https://wandb.ai/ma23m011-iit-madras/DL_A2/sweeps/tfufum48


wandb: Agent Starting Run: 6nu45mpx with config:
wandb: 	batch_size: 128
wandb: 	epochs: 10
wandb: 	freeze_up_to: 7
wandb: 	learning_rate: 0.001


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 219MB/s]


[earthy-sweep-1] Validation Accuracy: 76.25%


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▆▆▇▇▇▇███
train_loss,█▃▃▂▂▂▂▁▁▁
val_acc,▁
epoch,10
train_acc,75.9576
train_loss,0.71335
val_acc,76.25


...

epoch,▁▂▂▃▄▄▅▅▆▇▇█
train_acc,▁▅▆▆▆▆▆█▇█▇█
train_loss,█▄▄▃▃▃▃▁▂▁▁▁
val_acc,▁
epoch,12
train_acc,75.86759
train_loss,0.7306
val_acc,75.05


wandb: Agent Starting Run: bvsh5u0g with config:
wandb: 	batch_size: 32
wandb: 	epochs: 12
wandb: 	freeze_up_to: 0
wandb: 	learning_rate: 0.001


[chocolate-sweep-19] Validation Accuracy: 75.90%


epoch,▁▂▂▃▄▄▅▅▆▇▇█
train_acc,▁▅▅▅▆▆▆▇▇▇█▇
train_loss,█▄▃▃▃▃▂▁▁▁▁▁
val_acc,▁
epoch,12
train_acc,75.08751
train_loss,0.74159
val_acc,75.9


wandb: Agent Starting Run: rzpzqgf3 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	freeze_up_to: 0
wandb: 	learning_rate: 0.001
